In [20]:
import pandas as pd
import re
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
df = pd.read_csv('reviews.csv')


In [3]:
df.drop('Unnamed: 0', axis=1)


,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,x7mDIiDB3jEiPGPHOmDzyw,msQe1u7Z_XuqjGoqhB0J5g,iCQpiavjjPzJ5_3gPD5Ebg,2,2011-02-25,The pizza was okay. Not the best I've had. I p...,0,0,0
1,dDl8zu1vWPdKGihJrwQbpw,msQe1u7Z_XuqjGoqhB0J5g,pomGBqfbxcqPv14c3XH-ZQ,5,2012-11-13,I love this place! My fiance And I go here atl...,0,0,0
2,LZp4UX5zK3e-c5ZGSeo3kA,msQe1u7Z_XuqjGoqhB0J5g,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,Terrible. Dry corn bread. Rib tips were all fa...,3,1,1
3,Er4NBWCmCD4nM8_p1GRdow,msQe1u7Z_XuqjGoqhB0J5g,elqbBhBfElMNSrjFqW3now,2,2011-02-25,Back in 2005-2007 this place was my FAVORITE t...,2,0,0
4,jsDu6QEJHbwP2Blom1PLCA,msQe1u7Z_XuqjGoqhB0J5g,Ums3gaP2qM3W1XcA5r6SsQ,5,2014-09-05,Delicious healthy food. The steak is amazing. ...,0,0,0
5,pfavA0hr3nyqO61oupj-lA,msQe1u7Z_XuqjGoqhB0J5g,vgfcTvK81oD4r50NMjU2Ag,1,2011-02-25,This place sucks. The customer service is horr...,2,0,0
6,brokEno2n7s4vrwmmUdr9w,msQe1u7Z_XuqjGoqhB0J5g,AxeQEz3-s9_1TyIo-G7UQw,5,2011-10-10,"If you like Thai food, you have to try the ori...",1,0,0
7,kUZWBVZvhWuC8TWUg5AYyA,msQe1u7Z_XuqjGoqhB0J5g,zdE82PiD6wquvjYLyhOJNA,5,2012-04-18,AMAZING!!!\n\n I was referred here by a friend...,0,1,0
8,wcqt0III88LEcm19IxFFyA,msQe1u7Z_XuqjGoqhB0J5g,EAwh1OmG6t6p3nRaZOW_AA,4,2011-02-25,Ribs = amazing\n2 hour wait time= not so amazi...,0,0,0
9,LWUtqzNthMM3vpWZIFBlPw,msQe1u7Z_XuqjGoqhB0J5g,atVh8viqTj-sqDJ35tAYVg,2,2012-11-09,"Food is pretty good, not gonna lie. BUT you ha...",1,2,1


In [4]:
df = df.sample(frac=0.1)

In [8]:
def decide_pos_neg(stars):
    if stars < 3:
        return 1
    elif stars == 3:
        return -1
    else:
        return 0

In [10]:
df['pos_neg'] = df['stars'].apply(decide_pos_neg)

In [12]:
df = df[df['pos_neg'] != -1]

In [16]:
df.to_json('yelp_reviews.json')

In [17]:
X_train, X_test, y_train, y_test = train_test_split(df['text'].values, df['pos_neg'].values, test_size=0.2)

In [18]:
# converts a collection of text documents to a matrix of token counts
vectorizer = CountVectorizer(lowercase=True)

In [19]:
# Learn a vocabulary dictionary of all tokens in X_train (list of reviews)
vectorizer.fit_transform(X_train)

<7320x22731 sparse matrix of type '<class 'numpy.int64'>'
	with 474156 stored elements in Compressed Sparse Row format>

In [22]:
classifier = LogisticRegression()

In [23]:
# vectorizer.transform transforms X_train into a document term matrix
# this fits the linear regression model using least squares regression
classifier.fit(vectorizer.transform(X_train), y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [24]:
print("Score:", classifier.score(vectorizer.transform(X_test), y_test))

Score: 0.9360655737704918


In [25]:
classifier.predict(vectorizer.transform(["Terrible food, terrible service. Would never reccomend to a friend"]))


array([1])

In [26]:
classifier.predict(vectorizer.transform(X_test))

array([1, 1, 0, ..., 1, 0, 0])

In [27]:
y_test

array([1, 1, 0, ..., 0, 0, 0])